# Language Knowledge (Vocabulary)
Duration: 30 minutes
Content: This section tests your knowledge of Japanese vocabulary, including kanji readings, orthography, word formation, contextually-defined expressions, paraphrases, and usage
It mainly composes following five categories:
- ``Reading Kana`` (Pronunciation Questions): Given a kanji word, choose the correct kana reading.
- `Writing Kanji` (Writing Questions): Given a word written in kana, choose the correct kanji representation.
- `Word Meaning` Selection (Vocabulary Understanding): Choose the most suitable word to fill in the sentence from four options.
- `Synonym Replacement`: Select a word that has the same or similar meaning as the underlined word.
- `Vocabulary Usage`: Assess the usage of words in actual contexts, choosing the most appropriate word usage, including some common Japanese expressions or fixed phrases.

In [1]:
import pandas as pd
import json
import os
import pickle
import re
import uuid
from typing import *
from langchain_openai import AzureOpenAI,AzureChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import XinferenceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from IPython.display import display, Markdown, Latex
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage,RemoveMessage,HumanMessage,AIMessage,ToolMessage
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional

load_dotenv()

True

In [2]:
# Import N3 Vocabulary
file_path = '../Vocab/n3.csv'
# Read the CSV file
data = pd.read_csv(file_path)
words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
# Display the content of the CSV file
words.head()

,expression,reading
0,管,くだ
1,パーセント,パーセント
2,骨折,こっせつ
3,夜,よ
4,ペンキ,ペンキ


#### load Models

In [3]:
# azure_llm = AzureChatOpenAI(
#     azure_endpoint="https://tooldev-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview",
#     api_key=os.environ["AZURE_API_KEY"],
#     model_name="gpt-4o",
#     api_version="2025-01-01-preview",
#     temperature=0.5,
# )

In [4]:
aws_llm = ChatBedrock(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
     # model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    model_kwargs=dict(temperature=0.3),
    region = "us-east-2",
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
)

## Question Explore

# Kanji 读假名（读音问题）

In [5]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


# Graph state
class QuestionState(TypedDict):
    topic: str
    question: str
    improved_question: str
    final_question: str


kanji_example = """
問題 2

（ ）に入る最もよいものを、1・2・3・4から一つ選びなさい。

⸻

（問題例）

つくえの ( ) あります。
	1.	が
	2.	に
	3.	上
	4.	ペン

（解答のしかた）

正しい答えはこうなります。

つくえの 上 に ペン が あります。

⸻

問題

14.

山川大学では、（ ） 新入生がにアンケート調査を行っている。
	1.	大学生活
	2.	持っている
	3.	に対して
	4.	イメージ

15.

来週の夫の誕生日には、（ ） つもりだ。
	1.	最近
	2.	プレゼントする
	3.	かばんを
	4.	欲しがっている

16.

私は、健康の（ ）。
	1.	している
	2.	ために
	3.	毎日8時間以上寝る
	4.	ように

17.

部長が（ ） クッキーがとてもおいしいので、私も東京に行くことがあったら、買おうと思う。
	1.	たびに
	2.	ために
	3.	お土産の
	4.	ように

18.

私はこの図書館が好きだ。広くて本の数が多い（ ） いい。
	1.	景色を楽しみながら
	2.	大きな窓から街が見えて
	3.	だけでなく
	4.	読書ができるのも
"""

print(words)

# Nodes
def generate_question(state: QuestionState):
    """First LLM call to generate initial question"""

    msg = aws_llm.invoke(f"You are a japanese teacher. Your job is disordered sentences require selecting the correct arrangement order. Choose the correct option from the following 4 options. Only reference format, content cannot be referenced."
            f"The vocabulary should be restricted to N3 level, you can refer to the vocabulary in the word list, choosing random words for the questions"
            f"please refer the question examples following the formal exam paper"
            f"append the correct answer and explanation of main challenges and why teacher asks this question to candidate in chinese at each question"
            f"Finally, beautify markdown format"
            f"topic list: {state['topic']}"
            f"word list: {words}"
            f"formal exam paper: {kanji_example}")
    
    return {"question": msg.content}


# Build workflow
kanji_workflow = StateGraph(QuestionState)

# Add nodes
kanji_workflow.add_node("generate_question", generate_question)

# Add edges to connect nodes
kanji_workflow.add_edge(START, "generate_question")
kanji_workflow.add_edge("generate_question", END)

# Compile
kanji_graph = kanji_workflow.compile()


     expression reading
0             管      くだ
1         パーセント   パーセント
2            骨折    こっせつ
3             夜       よ
4           ペンキ     ペンキ
...         ...     ...
2134         大家     おおや
2135          分      ぶん
2136         盛り     さかり
2137         最中   さいちゅう
2138          谷      たに

[2139 rows x 2 columns]


In [6]:
# Show workflow
#display(Image(kanji_graph.get_graph().draw_png()))

In [7]:
# Invoke
kanji = kanji_graph.invoke({"topic": "店で価格を尋ねる | レストランで食べ物を注文する | 交通手段について話す| 天気の状況について話す | 趣味について話す | 店で商品の特徴や価格を尋ねる | スーパーマーケットで買い物をする | レストランで食べ物や飲み物を注文する | カフェでテイクアウトを頼む | コンビニで支払い方法について質問する |  ネットショッピングで商品を選ぶ"})
display(Markdown(kanji["question"]))

Error raised by bedrock service
Traceback (most recent call last):
  File "c:\Users\HW-BK003\Desktop\jlpt_simulator\.venv\Lib\site-packages\langchain_aws\llms\bedrock.py", line 956, in _prepare_input_and_invoke
    response = self.client.invoke_model(**request_options)
  File "c:\Users\HW-BK003\Desktop\jlpt_simulator\.venv\Lib\site-packages\botocore\client.py", line 570, in _api_call
    return self._make_api_call(operation_name, kwargs)
           ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HW-BK003\Desktop\jlpt_simulator\.venv\Lib\site-packages\botocore\context.py", line 124, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\HW-BK003\Desktop\jlpt_simulator\.venv\Lib\site-packages\botocore\client.py", line 1031, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.exceptions.ClientError: An error occurred (UnrecognizedClientException) when calling the InvokeModel operation: The security token included in the request is invalid.


ClientError: An error occurred (UnrecognizedClientException) when calling the InvokeModel operation: The security token included in the request is invalid.